# 1. Preliminaries

In [3]:
##!pip3 install qutip
import numpy as np
import qutip as qp
import scipy as si
import pandas as pd
from random import random   ## for generating a random number between 0 and 1

# 2. Gell_Mann matrix generator function

this function will generate the gellmann matrix of dimension d.

the function "gellmann(j,k,d)" will generate the jth_kth gellmann matrix of dimension d. so for every d dimension we have d^2 gellmann matrix. one of them is I, so the number of gellmann matrix that is not trivial is (d^2 - 1).

In [4]:
"""Generate generalized Gell-Mann matrices.
  .. module:: gellmann.py
     :synopsis: Generate generalized Gell-Mann matrices
  .. moduleauthor:: Jonathan Gross <jarthurgross@gmail.com>
"""
from itertools import product

def gellmann(j, k, d):
    r"""Returns a generalized Gell-Mann matrix of dimension d.
    
    According to the convention in *Bloch Vectors for Qubits* by Bertlmann and
    Krammer (2008), returns :math:`\Lambda^j` for :math:`1\leq j=k\leq d-1`,
    :math:`\Lambda^{kj}_s` for :math:`1\leq k<j\leq d`, :math:`\Lambda^{jk}_a`
    for :math:`1\leq j<k\leq d`, and :math:`I` for :math:`j=k=d`.
    Parameters
    ----------
    j : positive integer
        Index for generalized Gell-Mann matrix
    k : positive integer
        Index for generalized Gell-Mann matrix
    d : positive integer
        Dimension of the generalized Gell-Mann matrix
    Returns
    -------
    numpy.array
        A genereralized Gell-Mann matrix.
    """

    if j > k:
        gjkd = np.zeros((d, d), dtype=np.complex128)
        gjkd[j - 1][k - 1] = 1
        gjkd[k - 1][j - 1] = 1
    elif k > j:
        gjkd = np.zeros((d, d), dtype=np.complex128)
        gjkd[j - 1][k - 1] = -1.j
        gjkd[k - 1][j - 1] = 1.j
    elif j == k and j < d:
        gjkd = np.sqrt(2/(j*(j + 1)))*np.diag([1 + 0.j if n <= j
                                               else (-j + 0.j if n == (j + 1)
                                                     else 0 + 0.j)
                                               for n in range(1, d + 1)])
    else:
        gjkd = np.diag([1 + 0.j for n in range(1, d + 1)])

    return gjkd

def get_basis(d):
    r"""Return a basis of operators.
    
    The basis is made up of orthogonal Hermitian operators on a Hilbert space
    of dimension d, with the identity element in the last place.
    Parameters
    ----------
    d : int
        The dimension of the Hilbert space.
    Returns
    -------
    list of numpy.array
        The basis of operators.
    """
    return [gellmann(j, k, d) for j, k in product(range(1, d + 1), repeat=2)]

in this cell the gellmann matrix of 3 dimension will generate and they will save in a dictionary and labeled them by numbers 0,1,...,8 .

In [5]:
def gellmann3():
    gellmann3_basis = {}

    k=0
    for i in range(1,4):
        for j in range(1,4):
            gellmann3_basis.update({"G{}".format(k):gellmann(i,j,3)})
            k+=1
        
    return gellmann3_basis

# 3. Coefficients of gellmann's expansion

at this cell a random density matrix will generate by qutip library.

for calculating the coefficients of gellmann expansion we do this iteration:
1. calculate the tensor product of gellmann matrices for every coefficient
2. calculate the dot product of "ro" and the "gellmann's tensor product"
3. calculate the trace of this matrix and divide it into 4
4. append it to our index list

In [6]:
def coef_of_gellmann(ro):

    coef = {}  ## the coefficient's dictionary (suitable format for saving as a data)
    gellmann3_basis = gellmann3() ## the gellmann matrix for a 3 dimension system
    
    for i in range(9):
        for j in range(9):
            tensorP = np.kron(gellmann3_basis["G{}".format(i)],gellmann3_basis["G{}".format(j)]) ## Tensor product of gelllmann matrices
            c = np.trace(np.dot(ro,tensorP))/4   ## dot product and trace
            c = c.real       ## all of gellmann coefs is a real
            c = round(c,8)  
            coef.update({"a{}{}".format(i,j):c})  ## update the coef dict by a[i][j] and its gellmann coef for g[i][j]

    index = np.array(coef)  ## change list to np.array() object
    return(coef)

this cell is just a test of this algotithm in 3 dimension bipatite systems.

In [7]:
N = 3

ro = qp.rand_dm(N*N,0.5,dims=[[N,N], [N,N]])
coef_of_gellmann(ro)

{'a00': 0.0522061,
 'a01': -0.00490407,
 'a02': 0.00634005,
 'a03': -0.00654068,
 'a04': -0.02292012,
 'a05': -0.01468126,
 'a06': -0.01464795,
 'a07': -0.00364502,
 'a08': 0.01719028,
 'a10': -0.04022264,
 'a11': 0.03710369,
 'a12': -0.00768742,
 'a13': 0.02555521,
 'a14': 0.00309645,
 'a15': 0.00720487,
 'a16': 0.00043187,
 'a17': -0.00399608,
 'a18': 0.00436516,
 'a20': -0.00631644,
 'a21': 0.0115177,
 'a22': 0.00232515,
 'a23': 0.0057378,
 'a24': -0.02170102,
 'a25': -0.00769231,
 'a26': 0.02314279,
 'a27': -0.01268186,
 'a28': -0.00781535,
 'a30': -0.02496112,
 'a31': -0.00824812,
 'a32': -0.0226539,
 'a33': -0.04135151,
 'a34': 0.00122721,
 'a35': 0.00439332,
 'a36': -0.0405331,
 'a37': -5.91e-05,
 'a38': -0.02902844,
 'a40': 0.00278951,
 'a41': 0.03620197,
 'a42': 0.02254797,
 'a43': 0.00484,
 'a44': 0.02711497,
 'a45': 0.02181639,
 'a46': 0.01220908,
 'a47': -0.0127444,
 'a48': 0.04145063,
 'a50': 0.03477071,
 'a51': 0.02053101,
 'a52': -0.00563336,
 'a53': -0.0083802,
 'a54': 

# 4. PPT

In this cell we define a PPT funcion that check positive partial transpose.

In [8]:
def PPT(ro):
    """ positive partial transpose 
    input : density matrix
    out put:list of  coefficient of gellmann matrices and detect entangeled states
    """

    ro_pt = qp.partial_transpose(ro,[0,1])  ## partial transpose in subsystem 2
    eig = ro_pt.eigenstates()   ## calculate the eigenvalues and eigenstates
    eigv = [round(i,8) for i in eig[0]] 
    
    result = coef_of_gellmann(ro)
    
    if (eigv[0]<0 ):
            result.update({"label":1})  ## the entangled states that labeled by 1
            return result
    else :
            result.update({"label":2})  ## the unknown states that labeled by 2
            return result


in this cell we build 100 000 random states and labeled that. after that we build a data frame for saving and using in future.

In [18]:
#time

states={} #states
N = 3 #dimension

for i in range (1,100000+1):
    density = round(random(),4)     ## for build a random number between 0 and 1
    
    ro = qp.rand_dm(N*N, density, dims=[[N,N], [N,N]])     #generating random density matrix with a random density
    states.update({"{}".format(i):PPT(ro)})     # update the states by its number and label and gellmann coefs

df = pd.DataFrame(data=states).T  # build a data frame by pandas
df.head()

,a00,a01,a02,a03,a04,a05,a06,a07,a08,a10,...,a80,a81,a82,a83,a84,a85,a86,a87,a88,label
1,-0.050046,-0.001101,-0.006942,0.002876,0.048643,0.010141,-0.020011,0.017624,-0.062956,0.009758,...,-0.024817,0.019312,0.012277,-0.004409,-0.027848,0.017867,0.002271,-0.035783,0.25,1.0
2,0.004782,-0.015070,0.004060,-0.001692,0.007281,0.013864,0.015714,0.000917,-0.026297,0.024703,...,-0.025919,0.009750,-0.029247,0.000992,-0.033403,-0.016010,0.004529,0.041503,0.25,1.0
3,0.000000,0.000000,0.000000,0.000000,-0.144338,0.000000,0.000000,0.000000,0.125000,0.000000,...,-0.125000,0.000000,0.000000,0.000000,-0.072169,0.000000,0.000000,0.000000,0.25,2.0
4,-0.068928,0.000000,0.000000,0.000000,-0.023303,0.000000,0.000000,0.000000,-0.096482,0.000000,...,0.017761,0.027392,0.000000,0.015028,-0.016923,0.000000,0.000000,0.000000,0.25,1.0
5,0.046041,-0.045457,0.011715,0.017982,-0.004921,-0.007274,0.005969,0.007667,0.008899,0.002465,...,0.071236,0.002778,-0.008064,0.013210,-0.018381,0.007274,-0.002752,-0.007667,0.25,1.0


in this cell we save our data frame as a zip file

In [19]:
df.to_csv('data_of_states.csv.zip')
##df.to_csv('data_of_states.csv')